In [1]:
import pandas as pd

In [5]:
train_b = pd.read_excel('Train/train_bureau.xlsx')

In [6]:
train_d = pd.read_excel('Train/train_Data.xlsx')

In [7]:
test_b = pd.read_excel('Test/test_bureau.xlsx')

In [8]:
test_d = pd.read_excel('Test/test_Data.xlsx')